# Example Calculation
## Client: INTERNAL
---
## Project: develop hydraulic tools
## Calc: CALC-002, hydraulic sizing calculations
---
## Authentication
> Stamp, Permit
---
## Revision History
|Revision | Date | Description | By | Reviewer|
| :-------| :----|:------------|:---|:--------|
|       | | DRAFT | KCD |  |
|         |          |            |     |    |

---

## Introduction

In [37]:
# dependancies
using Pkg
#Pkg.add("DataFrames")
#Pkg.add("GridInterpolations")
#Pkg.add("Test")

# module is called Hydraulics2
Pkg.add(PackageSpec(url="https://github.com/kevindorma/Hydraulics2.jl"))

using DataFrames
#using GridInterpolations
#using Test
using CSV
using ExcelFiles
using Hydraulics2

#include("/Users/kevin/.julia/dev/Hydraulics/src/Hydraulics2.jl") # my hydraulic libary



  Updating git-repo `https://github.com/kevindorma/Hydraulics2.jl`
  Updating git-repo `https://github.com/kevindorma/Hydraulics2.jl`
 Resolving package versions...
  Updating `~/.julia/environments/v1.3/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.3/Manifest.toml`
 [no changes]


List a common utility table.

In [38]:
Hydraulics2.IDmm

,NPS,Schedule,Idmm
,Float64,String,Float64
1,0.125,OD,10.287
2,0.25,OD,13.716
3,0.375,OD,17.145
4,0.5,OD,21.336
5,0.75,OD,26.67
6,1.0,OD,33.401
7,1.25,OD,42.164
8,1.5,OD,48.26
9,2.0,OD,60.325


## Format of input data
Input data is required the following tables.
* fluidList: this tabulates fluid density and viscosity for a given fluid. Used for sizing and hydraulic calculations.

Sizing
* Put the sizing information in the table lineSizing. A spreadsheet is very convenient becauase it allows one to use descriptors for properties (such as fluid type, pipe material, pipe schedule).

Hydraulics
* Need a table (lines) for describing overall conditions for each line segment.
* also need a table (fittingList) for listing all of the piping components in each line segment.

A spreadsheet is very convenient for tabulating the input data. Dropdown menus are used to ensure inputs are correct (fluid type, NPS, pipe schedule). Then the hydraulic calculations are performed using the input data in the spreadsheet.



In [39]:
# these are the specific data
lineSizing = DataFrame(load("sizingHydraulicSample_rev_1.xlsx", "sizing"))
#lineHydraulics = DataFrame(load("sizingHydraulicSample_rev_1.xlsx", "hydraulics"))
fluidList = DataFrame(load("sizingHydraulicSample_rev_1.xlsx", "fluidList"))
#fittingList = DataFrame(load("sizingHydraulicSample_rev_1.xlsx", "fittingList"));

## Check input tables for inconsistancies

In [40]:
Hydraulics2.checkLineList(lineSizing,fluidList)

,message,entry,Segment,NPS,Schedule,material,fluidName
,String,Int64,String,Float64,String,String,String


In [41]:
Hydraulics2.pipeRoughness;

## Line Sizing

The spreadsheet hydraulicDescription_2.xls contains the data needed for line sizing.

In [42]:
myLineSize = Hydraulics2.getLineSize(lineSizing, fluidList)

,Segment,mmDP100,mmErosion,mmNeeded,Schedule
,String,Float64,Float64,Float64,String
1,A,295.871,318.281,318.281,STD
2,B,66.0907,60.8437,66.0907,STD
3,C,53.1497,21.183,53.1497,STD
4,D,43.9973,27.3472,43.9973,STD


## Line Sizing

Get the next larger line sizes from the NPS list.

In [43]:
chosenLineSizes = Hydraulics2.getListLargerNPS(myLineSize)

,Segment,NPS,Schedule,IDmm
,String,Float64,String,Float64
1,A,14.0,STD,336.54
2,B,3.0,STD,77.9
3,C,2.5,STD,62.6
4,D,2.0,STD,52.5


## Bump up a line size

The NPS 2.5 is not common in many industries. We will bump line 3 to the next larger size.

In [44]:
Hydraulics2.incrementLineSize(3,chosenLineSizes,1)
chosenLineSizes

,Segment,NPS,Schedule,IDmm
,String,Float64,String,Float64
1,A,14.0,STD,336.54
2,B,3.0,STD,77.9
3,C,3.0,STD,77.9
4,D,2.0,STD,52.5


In [45]:
combined1=outerjoin(lineSizing, chosenLineSizes, on = intersect(names(lineSizing), names(chosenLineSizes)), makeunique=false,indicator=nothing,validate=(false,false))
combined2=outerjoin(combined1, myLineSize, on = intersect(names(combined1), names(myLineSize)), makeunique=false,indicator=nothing,validate=(false,false))

,Segment,Description,LineTag,PnID,fluidName,Schedule,Material,massFlow
,String?,String?,String?,String?,String?,String?,String?,Float64?
1,A,Test line,L-001,PID-007,propane,STD,Commercial steel,500000.0
2,B,Testline2,L-002,PID-007,water,STD,Commercial steel,66000.0
3,C,Testline3,L-003,PID-007,water,STD,Commercial steel,6000.0
4,D,Testline3,L-004,PID-007,water,STD,Commercial steel,10000.0


In [46]:
names(combined2)

19-element Array{String,1}:
 "Segment"    
 "Description"
 "LineTag"    
 "PnID"       
 "fluidName"  
 "Schedule"   
 "Material"   
 "massFlow"   
 "margin"     
 "frictionCsi"
 "kPaPer100m" 
 "rho_kgm3"   
 "mu_mPas"    
 "roughnessMM"
 "NPS"        
 "IDmm"       
 "mmDP100"    
 "mmErosion"  
 "mmNeeded"   

In [47]:
resultTable1 = select(combined2, [:Segment, :fluidName, :massFlow, :margin, :mmDP100, :mmErosion, :NPS, :Schedule])

,Segment,fluidName,massFlow,margin,mmDP100,mmErosion,NPS,Schedule
,String?,String?,Float64?,Float64?,Float64?,Float64?,Float64?,String?
1,A,propane,500000.0,1.5,295.871,318.281,14.0,STD
2,B,water,66000.0,1.0,66.0907,60.8437,3.0,STD
3,C,water,6000.0,1.0,53.1497,21.183,3.0,STD
4,D,water,10000.0,1.0,43.9973,27.3472,2.0,STD


In [48]:
CSV.write("exportSampleSizingTable.csv", combined2)

"exportSampleSizingTable.csv"

## Dump the results to preliminary files for hydraulic calculations

Write the sizing table to a CSV file. Then create a list of sample fittings (100 m of pipe) for each segment. These CSV files can be imported into a spreadsheet to produce a complete hydraulic input sheet.


In [49]:
sampleHydraulicDF = Hydraulics2.sizing2hydraulics(lineSizing, chosenLineSizes)

,Segment,Description,LineTag,PnID,NPS,Schedule,Material,fluidName
,String,String,String,String,Float64,String,String,Any
1,A,Test line,L-001,PID-007,14.0,STD,Commercial steel,propane
2,B,Testline2,L-002,PID-007,3.0,STD,Commercial steel,water
3,C,Testline3,L-003,PID-007,3.0,STD,Commercial steel,water
4,D,Testline3,L-004,PID-007,2.0,STD,Commercial steel,water


In [50]:
# dump the sample hydraulic file to a CSV file
CSV.write("exportSampleHydraulicTable.csv", sampleHydraulicDF)

"exportSampleHydraulicTable.csv"

In [51]:
sampleFittingDF = Hydraulics2.sizing2fittingList(lineSizing, chosenLineSizes)

,Segment,fittingType,num_length_m,comment,revision
,String,String,Float64,String,String
1,A,PIPE,100.0,from line sizing,A
2,B,PIPE,100.0,from line sizing,A
3,C,PIPE,100.0,from line sizing,A
4,D,PIPE,100.0,from line sizing,A


In [52]:
# dump the sample fitting list to a CSV file
CSV.write("exportSampleFittingTable.csv", sampleFittingDF)

"exportSampleFittingTable.csv"